In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import numpy as np

# Load the dataset with 'latin1' encoding
df = pd.read_csv('spam_dataset.csv', encoding='latin1')

# Split the dataset into features and labels
X = df['sms']
y = df['label']


In [ ]:


# Convert text data into numerical data using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X).toarray()  # Convert sparse matrix to dense array


In [ ]:
# Use SMOTE to handle imbalanced dataset
# from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

NameError: name 'SMOTE' is not defined

In [5]:
from torch import nn, optim, tensor
from torch.nn import functional as F
import time

import pandas
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from torch import nn, optim, tensor
from torch.nn import functional as F


In [14]:
class Classifier(nn.Module):
    def __init__(self, embedding_dim, num_labels, dropout):
        super(Classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_labels = num_labels
        self.dropout = dropout

        self.dp = nn.Dropout(self.dropout)
        self.ff = nn.Linear(self.embedding_dim, self.num_labels)

    def forward(self, input_embeddings):
        tensor = self.dp(input_embeddings)
        tensor = self.ff(tensor)
        return tensor, F.softmax(tensor, dim=-1)


class Batcher(object):
    def __init__(self, data_x, data_y, batch_size):
        self.data_x = data_x
        self.data_y = data_y
        self.batch_size = batch_size
        self.n_samples = data_x.shape[0]
        self.indices = torch.randperm(self.n_samples)
        self.ptr = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.ptr > self.n_samples:
            self.ptr = 0
            self.indices = torch.randperm(self.n_samples)
            raise StopIteration
        else:
            batch_indices = self.indices[self.ptr:self.ptr+self.batch_size]
            self.ptr += self.batch_size
            return self.data_x[batch_indices], self.data_y[batch_indices]


data_df = pandas.read_csv("/content/data-en-hi-de-fr.csv")

data_df.dropna(inplace=True)
data_df.drop_duplicates(inplace=True)
data_df.rename(columns={
    "Category": "labels",
    "Message": "text"
}, inplace=True)

le = LabelEncoder()
le.fit(data_df.labels)
data_df["labels"] = le.transform(data_df.labels)

train_x, test_x, train_y, test_y = \
    train_test_split(data_df.text, data_df.labels, stratify=data_df.labels, test_size=0.15,
                     random_state=123)

train_x_de, test_x_de, train_y_de, test_y_de = \
    train_test_split(data_df.text_hi, data_df.labels, stratify=data_df.labels, test_size=0.15,
                     random_state=123)

sentences = train_x.tolist()
test_sentences = test_x_de.tolist()

labels = torch.tensor(train_y.tolist())
test_labels = torch.tensor(test_y_de.tolist())

# encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoder = SentenceTransformer('quora-distilbert-multilingual')
print('Encoding segments...')
start = time.time()
embedding = encoder.encode(sentences, convert_to_tensor=True)
test_sentences_embedding = encoder.encode(test_sentences, convert_to_tensor=True)
print(f"Encoding completed in {time.time() - start} seconds.")

train_batcher = Batcher(embedding, labels, batch_size=16)

num_samples, embeddings_dim = embedding.size()
n_labels = labels.unique().shape[0]

classifier = Classifier(embeddings_dim, n_labels, dropout=0.01)

optimizer = optim.Adam(classifier.parameters())
loss_fn = nn.CrossEntropyLoss()

for e in range(10):
    total_loss = 0
    for batch in train_batcher:
        x, y = batch
        optimizer.zero_grad()
        model_output, prob = classifier(x)
        loss = loss_fn(model_output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'epoch:{e}, total_loss:{total_loss}')

with torch.no_grad():
    model_output, prob = classifier(test_sentences_embedding)
    predictions = torch.argmax(prob, dim=-1)
    results = classification_report(predictions, test_labels)
    print(results)

Encoding segments...
Encoding completed in 353.12576961517334 seconds.
epoch:0, total_loss:60.209235444664955
epoch:1, total_loss:34.170039696618915
epoch:2, total_loss:26.798776485025883
epoch:3, total_loss:23.24302877113223
epoch:4, total_loss:20.70875272154808
epoch:5, total_loss:19.180852215271443
epoch:6, total_loss:17.432323425775394
epoch:7, total_loss:16.62215864774771
epoch:8, total_loss:15.740114335436374
epoch:9, total_loss:14.825828957837075
              precision    recall  f1-score   support

           0       1.00      0.93      0.96       727
           1       0.46      0.94      0.62        47

    accuracy                           0.93       774
   macro avg       0.73      0.93      0.79       774
weighted avg       0.96      0.93      0.94       774



In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics import classification_report
import time
from sentence_transformers import SentenceTransformer
import numpy as np

class Classifier(nn.Module):
    def __init__(self, embedding_dim, num_labels, dropout):
        super(Classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.num_labels = num_labels
        self.dropout = dropout

        self.dp = nn.Dropout(self.dropout)
        self.ff = nn.Linear(self.embedding_dim, self.num_labels)

    def forward(self, input_embeddings):
        tensor = self.dp(input_embeddings)
        tensor = self.ff(tensor)
        return tensor, F.softmax(tensor, dim=-1)

class Batcher(object):
    def __init__(self, data_x, data_y, batch_size):
        self.data_x = data_x
        self.data_y = data_y
        self.batch_size = batch_size
        self.n_samples = data_x.shape[0]
        self.indices = torch.randperm(self.n_samples)
        self.ptr = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.ptr >= self.n_samples:
            self.ptr = 0
            self.indices = torch.randperm(self.n_samples)
            raise StopIteration
        else:
            batch_indices = self.indices[self.ptr:self.ptr+self.batch_size]
            self.ptr += self.batch_size
            return self.data_x[batch_indices], self.data_y[batch_indices]


data_df = pd.read_csv("/content/data-en-hi-de-fr.csv")

data_df.dropna(inplace=True)
data_df.drop_duplicates(inplace=True)
data_df.rename(columns={
    "Category": "labels",
    "Message": "text"
}, inplace=True)

le = LabelEncoder()
le.fit(data_df.labels)
data_df["labels"] = le.transform(data_df.labels)

train_x, test_x, train_y, test_y = \
    train_test_split(data_df.text, data_df.labels, stratify=data_df.labels, test_size=0.15,
                     random_state=123)

train_x_de, test_x_de, train_y_de, test_y_de = \
    train_test_split(data_df.text, data_df.labels, stratify=data_df.labels, test_size=0.15,
                     random_state=123)

sentences = train_x.tolist()
test_sentences = test_x_de.tolist()

labels = train_y.to_numpy()
test_labels = test_y_de.to_numpy()

# encoder = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoder = SentenceTransformer('quora-distilbert-multilingual')
print('Encoding segments...')
start = time.time()
embedding = encoder.encode(sentences, convert_to_tensor=True)

# Applying SMOTE to the embeddings
smote = SMOTE(random_state=42)
oversampled_embeddings, oversampled_labels = smote.fit_resample(embedding, labels)

test_sentences_embedding = encoder.encode(test_sentences, convert_to_tensor=True)
print(f"Encoding completed in {time.time() - start} seconds.")

train_batcher = Batcher(torch.tensor(oversampled_embeddings), torch.tensor(oversampled_labels), batch_size=16)

num_samples, embeddings_dim = embedding.size()
n_labels = torch.tensor(oversampled_labels).unique().shape[0]

classifier = Classifier(embeddings_dim, n_labels, dropout=0.01)

optimizer = optim.Adam(classifier.parameters())
loss_fn = nn.CrossEntropyLoss()

for e in range(10):
    total_loss = 0
    for batch in train_batcher:
        x, y = batch
        optimizer.zero_grad()
        model_output, prob = classifier(x)
        loss = loss_fn(model_output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'epoch:{e}, total_loss:{total_loss}')

with torch.no_grad():
    model_output, prob = classifier(test_sentences_embedding)
    predictions = torch.argmax(prob, dim=-1)
    results = classification_report(predictions, test_labels)
    print(results)


Encoding segments...
Encoding completed in 350.3312654495239 seconds.
epoch:0, total_loss:117.23487184382975
epoch:1, total_loss:55.81252033263445
epoch:2, total_loss:42.591582499444485
epoch:3, total_loss:35.13796570757404
epoch:4, total_loss:30.69702981505543
epoch:5, total_loss:27.116019108332694
epoch:6, total_loss:24.74434843636118
epoch:7, total_loss:23.39571693399921
epoch:8, total_loss:21.975981851806864
epoch:9, total_loss:20.27736062789336
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       667
           1       0.94      0.84      0.89       107

    accuracy                           0.97       774
   macro avg       0.96      0.92      0.93       774
weighted avg       0.97      0.97      0.97       774

